<a href="https://colab.research.google.com/github/emmanuelinfante/SubtitlesEveryone/blob/main/SubtitlesForEveryone(Spanish).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**📝¡SubtitlesForEveryone!** Transcribe y traduce de forma gratuita.

¡Bienvenido al notebook oficial de **SubtitlesForEveryone**!

>Este Notebook, recopila y facilita el uso de herramientas de inteligencia artificial para la transcripcion y traduccion de audio. Si necesitas crear subtitulos de tu pelicula o de un video, solo tienes que seguir las indicaciones que te proporcionaré a continuación, y comprobarás lo fácil que es.

## ¿Cómo empezar? 🚀

1. **Habilita la GPU**: Para obtener una transcripción rápida, asegúrate de habilitar la GPU. Ve a "Entorno de ejecución" > "Cambiar tipo de entorno de ejecución" > y selecciona "T4 GPU" en la opción "Acelerador de hardware".
2. **Sube tu audio o Pega Una URL de Youtube**: Usa la herramienta de abajo para subir tu archivo de audio o usa la herramienta para importar tu video de youtube.
3. **Ejecuta las celdas**: Simplemente ejecuta las celdas en orden, ¡y verás la magia suceder! ✨

**Nota**: Si eres nuevo en Google Colab, cada celda con código se ejecuta haciendo clic en el botón de reproducción (▶️) a la izquierda de la celda, o puedes presionar `Shift + Enter`.


# **1. ¡Instalación de paquetes necesarios! 📦**
Primero, necesitamos descargar todas las librerias necesarias para hacer funcionar este notebook!

In [ ]:
#Instalacion de paquetes livianos pero necesarios
!pip install srtranslator
!pip3 install ip-rotator
!pip install yt-dlp
!pip install srt
!pip install translators

In [ ]:
#Instalacion de paquetes fundamentales
!pip install demucs
!pip install git+https://github.com/m-bain/whisperx

# **Ahora si, Crea tu subtitulo!** 📜

In [ ]:
#@markdown # **1. Sube tu archivo de audio 📻**
#@markdown Haz clic en el botón de abajo para subir tu archivo de audio. Asegúrate de que sea un archivo en formato MP3.
from google.colab import files

uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

# Renombra el archivo de audio
import os
os.rename(audio_file, "audio.mp3")

In [ ]:
import sys
import warnings
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

Type = "Youtube video or playlist"
#@markdown # Descarga el audio de su video de youtube 📀
URL = "" #@param {type:"string"}
#@markdown Tardara unos segundos en cargar, porfavor, sea paciente.

video_path_local_list = []

if Type == "Youtube video or playlist":

  ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'outtmpl': 'audio',  # Rename the extracted MP3 file to "audio.mp3"
    'postprocessors': [{
      'key': 'FFmpegExtractAudio',
      'preferredcodec': 'mp3',
    }]
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download([URL])
    list_video_info = [ydl.extract_info(URL, download=False)]

#change sample rate
import librosa
import soundfile as sf

def convertir_frecuencia(archivo_entrada, archivo_salida):
  # Cargar el archivo de audio
  senal, frecuencia_muestreo = librosa.load(archivo_entrada)

  # Convertir a mono
  senal_mono = librosa.to_mono(senal)

  # Nueva frecuencia de muestreo
  nueva_frecuencia = 16000

  # Convertir la señal a la nueva frecuencia
  senal_convertida = librosa.resample(senal, orig_sr=frecuencia_muestreo, target_sr=nueva_frecuencia)

  # Guardar el archivo de audio convertido
  sf.write(archivo_salida, senal_convertida, nueva_frecuencia)

# Ejemplo de uso
archivo_entrada = "audio.mp3"
archivo_salida = "audio_16_000.mp3"

convertir_frecuencia(archivo_entrada, archivo_salida)

In [ ]:
#@markdown # *Extraer las vocales del audio*
#@markdown (Opcional, pero <font color=4bb340> **Altamente Recomendable** <font color=d5d5d5> usarlo):
#@markdown - ¡Gran Mejora En Idiomas Orientales!
#@markdown - Evita que WhisperX Confunda Sonidos de Fondo con el Habla.
#@markdown - Mejora la transcripcion con cualquier idioma.
#@markdown - Gracias a esta extraccion, sera muy raro e improbable que tengas errores en tu transcripcion.

#@markdown **Ahora funciona con audios mayores de 30 minutos!** No importa la duracion, puedes usar audios incluso de 10 horas!

import os
import librosa
from pydub import AudioSegment
import demucs.separate
import shutil

# Definir la duración máxima en segundos (30 minutos)
max_duration = 30 * 60

# Cargar el audio
audio_path = '/content/audio_16_000.mp3'
audio = AudioSegment.from_file(audio_path)

# Obtener la duración del audio en segundos
duration = librosa.get_duration(filename=audio_path)

# Si la duración es mayor que la duración máxima, dividir el audio
if duration > max_duration:
    # Calcular el número de segmentos
    num_segments = int(duration // max_duration) + 1

    # Dividir el audio en segmentos y guardar cada segmento como un nuevo archivo de audio
    for i in range(num_segments):
        start_time = i * max_duration
        end_time = (i + 1) * max_duration if (i + 1) * max_duration < duration else duration
        segment = audio[start_time * 1000:end_time * 1000]  # pydub trabaja en milisegundos
        segment_path = f'{os.path.splitext(audio_path)[0]}_segment{i}{os.path.splitext(audio_path)[1]}'
        segment.export(segment_path, format='wav')

        # Aplicar demucs a cada archivo de audio
        demucs.separate.main(["-d", "cuda", "--mp3", "--mp3-bitrate", "96", "--two-stems=vocals", "-n", "hdemucs_mmi", segment_path, "--out", "/content", "--mp3-preset", "7"])

    # Unir todos los resultados en un solo archivo de audio
    # Definir la ruta de la carpeta que contiene los segmentos de audio
    folder_path = '/content/hdemucs_mmi'

    # Obtener una lista de todas las carpetas en la carpeta de segmentos
    folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

    # Ordenar la lista de carpetas para asegurarse de que los segmentos se unan en el orden correcto
    # Aquí asumimos que el nombre de la carpeta termina con el número de segmento
    folders.sort(key=lambda x: int(x.rsplit('segment', 1)[-1]))

    # Crear un objeto AudioSegment vacío para almacenar el audio combinado
    combined = AudioSegment.empty()

    # Para cada carpeta, buscar el archivo "vocals.mp3"
    for folder in folders:
        vocals_file = os.path.join(folder_path, folder, 'vocals.mp3')
        if os.path.isfile(vocals_file):
            # Cargar el audio y agregarlo a un objeto AudioSegment combinado
            audio_segment = AudioSegment.from_mp3(vocals_file)
            combined += audio_segment

    # Exportar el objeto AudioSegment combinado a un archivo de audio mp3
    combined.export('/content/result/vocals.mp3', format='mp3')

    # Borrar la carpeta hdemucs_mmi y todo su contenido
    shutil.rmtree('/content/hdemucs_mmi')
else:
    # Si la duración es menor o igual a la duración máxima, aplicar demucs directamente al audio
    demucs.separate.main(["-d", "cuda", "--mp3", "--mp3-bitrate", "96", "--two-stems=vocals", "-n", "hdemucs_mmi", audio_path, "--out", "/content", "--mp3-preset", "7"])

    # Mover el archivo de audio resultante a la carpeta de resultados
    output_path = '/content/hdemucs_mmi/audio_16_000/vocals.mp3'
    if os.path.isfile(output_path):
        shutil.move(output_path, '/content/result/vocals.mp3')
        # Borrar la carpeta hdemucs_mmi y todo su contenido
        shutil.rmtree('/content/hdemucs_mmi')

In [ ]:
#@markdown # **2. Transcripción De Voz ✍️**
#@markdown Ahora, vamos a transcribir tu archivo de audio.
import whisperx
import gc

audio_file = "/content/result/vocals.mp3" #@param ['/content/result/vocals.mp3', '/content/audio_16_000.mp3']
from_language= 'es' #@param ['af', 'am', 'ar', 'as', 'az', 'ba', 'be', 'bg', 'bn', 'bo', 'br', 'bs', 'ca', 'cs', 'cy', 'da', 'de', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fo', 'fr', 'gl', 'gu', 'ha', 'haw', 'he', 'hi', 'hr', 'ht', 'hu', 'hy', 'id', 'is', 'it', 'ja', 'jw', 'ka', 'kk', 'km', 'kn', 'ko', 'la', 'lb', 'ln', 'lo', 'lt', 'lv', 'mg', 'mi', 'mk', 'ml', 'mn', 'mr', 'ms', 'mt', 'my', 'ne', 'nl', 'nn', 'no', 'oc', 'pa', 'pl', 'ps', 'pt', 'ro', 'ru', 'sa', 'sd', 'si', 'sk', 'sl', 'sn', 'so', 'sq', 'sr', 'su', 'sv', 'sw', 'ta', 'te', 'tg', 'th', 'tk', 'tl', 'tr', 'tt', 'uk', 'ur', 'uz', 'vi', 'yi', 'yo', 'zh']{allow-input: true}
device = "cuda"
chunk_size = 10
batch_size = 16
compute_type = "float16"
beam_size = 10
no_align = True
print_progress = True

model = whisperx.load_model("large-v3", device, compute_type=compute_type)
audio = whisperx.load_audio(audio_file)

# Transcription
result = model.transcribe(
    audio,
    batch_size=batch_size,
    chunk_size=chunk_size,
    language=from_language,
    print_progress=print_progress
)
print(result["segments"])

#srt
from whisperx.utils import WriteSRT

audio_filename = audio_file.split("/")[-1]
srt_filename = audio_filename.split(".")[0] + ".srt"

with open(srt_filename, "w", encoding="utf-8") as srt:
    writesrt=WriteSRT(".")  #output file directory
    writesrt.write_result(result=result, file=srt,options={"max_line_width":None,"max_line_count":None,"highlight_words":False,"chunk_size":10})

#vtt
from whisperx.utils import WriteSRT

audio_filename = audio_file.split("/")[-1]
srt_filename = audio_filename.split(".")[0] + ".vtt"

with open(srt_filename, "w", encoding="utf-8") as srt:
    writesrt=WriteSRT(".")  #output file directory
    writesrt.write_result(result=result, file=srt,options={"max_line_width":None,"max_line_count":None,"highlight_words":False,"chunk_size":10})

#@markdown - Si deseas que WhisperX detecte automáticamente el idioma, simplemente debes dejar vacío el campo "from_language", es decir, no debe contener ningún carácter, para la detección automática.

#@markdown Para usar unicamente las vocales: _**/content/result/vocals.mp3**_

#@markdown Para usar el audio original: _**/content/audio_16_000.mp3**_

### Download vocals.vtt 📂
Download_vocals_vtt = False # @param{type:"boolean"}
if Download_vocals_vtt:
 from google.colab import files
 files.download ("vocals.vtt")
### Download vocals.srt 📂
Download_vocals_srt = False # @param{type:"boolean"}
if Download_vocals_srt:
 from google.colab import files
 files.download ("vocals.srt")

### Download audio_16_000.vtt 📂
Download_audio_16_000_vtt = False # @param{type:"boolean"}
if Download_audio_16_000_vtt:
 from google.colab import files
 files.download ("audio_16_000.vtt")
### Download audio_16_000.srt 📂
Download_audio_16_000_srt = False # @param{type:"boolean"}
if Download_audio_16_000_srt:
 from google.colab import files
 files.download ("audio_16_000.srt")

In [ ]:
#@markdown # **3. Traduzca Su Subtitulo! 🌍**
#@markdown Traduce tu subtítulo y tradúcelo a otro idioma. Puedes usar más de 6 traductores para elegir.
import ip_rotator
proxy = ip_rotator.Proxy(https = True)
import translators
import srt

def get_user_input():
  """Prompts the user to choose the SRT file and returns the filename."""
  while True:
    filename = input("Enter the SRT filename (audio_16_000.srt or vocals.srt): ")
    if filename in ("audio_16_000.srt", "vocals.srt"):
      return filename
    else:
      print("Invalid filename. Please choose 'audio_16_000.srt' or 'vocals.srt'.")

# Read the original SRT file based on user input
filename = get_user_input()
base_filename, _ = os.path.splitext(filename)  # Extract base filename without extension

with open(filename, "r", encoding="utf-8") as f:
  subtitles = srt.parse(f.read())

# Function to translate a subtitle (consider error handling)
def translate_subtitle(subtitle):
  try:
    translated_content = translators.translate_text(
        query_text=subtitle.content,
        from_language="auto"#@param {type:"string"}
        ,
        to_language="en"#@param {type:"string"}
        ,
        translator="google" #@param ['alibaba', 'apertium', 'argos', 'baidu', 'bing', 'caiyun', 'cloudTranslation', 'elia', 'google', 'hujiang', 'iciba', 'iflytek', 'iflyrec', 'itranslate', 'judic', 'languageWire', 'lingvanex', 'niutrans', 'mglip', 'mirai', 'modernMt', 'myMemory', 'papago', 'qqFanyi', 'qqTranSmart', 'reverso', 'sogou', 'sysTran', 'tilde', 'translateCom', 'translateMe', 'utibet', 'volcEngine', 'yandex', 'yeekit', 'youdao']{allow-input: true}
    )
    print(f" Subtitle Results {subtitle.index}:")
    print(f"  Original: {subtitle.content}")
    print(f"  Translation: {translated_content}")
    return srt.Subtitle(subtitle.index, subtitle.start, subtitle.end, translated_content)
  except Exception as e:  # Catch potential translation errors
    print(f"Error translating subtitle {subtitle.index}: {e}")
    return subtitle  # Return the original subtitle if translation fails

# Translate each subtitle
translated_subtitles = [translate_subtitle(subtitle) for subtitle in subtitles]

# Write the translated subtitles to a new file with modified name
translated_filename = f"{base_filename}_translated_by_translators.srt"
with open(translated_filename, "w", encoding="utf-8") as f:
  f.write(srt.compose(translated_subtitles))

### Download the translated SRT file (optional)
Download_srt = False  # Set to True to download the file

if Download_srt:
  from google.colab import files  # Assuming you're using Google Colab
  files.download(translated_filename)

### Download result srt 📂
Download_audio_16_000_srt = False # @param{type:"boolean"}
if Download_audio_16_000_srt:
 from google.colab import files
 files.download ("audio_16_000_translated_by_translators.srt")

### Download result srt 📂
Download_vocals_srt = False # @param{type:"boolean"}
if Download_vocals_srt:
 from google.colab import files
 files.download ("vocals_translated_by_translators.srt")

#@markdown Subtitulo De Vocales: _**/content/vocals.srt**_

#@markdown Subtitulo Del Audio Original: _**/content/audio_16_000.srt**_

In [ ]:
#@markdown # <font color=somblue>**DeepL Unlimited Subtitles!** <font color=d5d5d5> *Best Translator.*
#@markdown ¡Traduce tus subtítulos con gran precisión usando DeepL de forma ilimitadamente!
Source_Language = "auto" # @param ['ar', 'auto', 'bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'id', 'it', 'ja', 'ko', 'lt', 'lv', 'nb', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sv', 'tr', 'uk', 'zh']{allow-input: true}

#@title Target Language
Target_Language = "en" #@param ['ar', 'bg', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fi', 'fr', 'hu', 'id', 'it', 'ja', 'ko', 'lt', 'lv', 'nb', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sv', 'tr', 'uk', 'zh']{allow-input: true}

import os

# SRT File
from srtranslator import SrtFile

from srtranslator.translators.pydeeplx import PyDeepLX
translator = PyDeepLX()
filepath = "/content/vocals.srt" #@param ['/content/audio_16_000.srt','/content/vocals.srt']

# SRT File
sub = SrtFile(filepath)
# Translate
sub.translate(translator, Source_Language, Target_Language)

# Making the result subtitles prettier
sub.wrap_lines()
sub.save(f"{os.path.splitext(filepath)[0]}_translated_by_DeepL.srt")

### Download result 📂
Download_vocal_srt = False # @param{type:"boolean"}
if Download_vocal_srt:
 from google.colab import files
 files.download ("vocals_translated_by_DeepL.srt")

### Download result 📂
Download_audio_srt = False # @param{type:"boolean"}
if Download_audio_srt:
 from google.colab import files
 files.download ("audio_16_000_translated_by_DeepL.srt")

#@markdown Para subtítulos con vocales: _**/content/vocals.srt**_

#@markdown Para subtítulos del audio original: _**/content/audio_16_000.srt**_

#@markdown **Advertencia:**
#@markdown *El uso de este traductor garantizará traducciones muy buenas, pero no es rápido. La duración del proceso de traducción depende de la longitud de sus subtítulos para traducir.*


## **Lenguajes disponibles para DeepL.** 🗨️


```
'ar': Arabic
'auto': Auto
'bg': Bulgaria
'cs': Czech
'da': Danish
'de': German
'el': Greek
'en': English
'es': Spanish
'et': Estonian
'fi': Finnish
'fr': French
'hu': Hungarian
'id': Indonesian
'it': Italian
'ja': Japanese
'ko': Korean
'lt': Lithuanian
'lv': Latvian
'nb': Norwegian Bokmål
'nl': Dutch
'pl': Polish
'pt': Portuguese
'ro': Romanian
'ru': Russian
'sk': Slovak
'sl': Slovenian
'sv': Swedish
'tr': Turkish
'uk': Ukrainian
'zh': Chinese
```



## **Créditos y Agradecimientos Especiales**
Este notebook, combina muchas librerias gratuitas provenientes de Github y PyPI para lograr hacer su cometido. A continuacion, compartire cuales son:

- [**WhisperX** - Inteligencia Artificial Para transcribir](https://github.com/m-bain/whisperX)
- [**Translators** - Librebria con Api para mas de 10 idiomas](https://pypi.org/project/translators/)
- [**Srt** - Necesario para leer Archivos .srt](https://pypi.org/project/srt/)
- [**Demucs** - Para la extracion de voz](https://pypi.org/project/demucs/)
- [**Ctranslate2** - Necesario para WhisperX](https://pypi.org/project/ctranslate2/)
- [**Yt-dlp** - para la descarga de videos](https://pypi.org/project/yt-dlp/)
- [**Ip-Rotator** - Proxy para Apis](https://pypi.org/project/ip-rotator/)
- [**Srtranslator** - Traduccion De Subtitulos Con Deepl](https://pypi.org/project/srtranslator/)

Aclaracion Importante, este notebook tambien utiliza codigo propio para la creacion de subtitulos; Translators **No fue creado para traducir subtitulos, tuve que escribir yo el codigo para que funcionara de esa forma.** Aun asi, ¡Agradezco enormemente el equipo detrás de todos estos proyectos, son importantes para hacer posible este proyecto! Sin nada mas que decir, Disfruta de este proyecto!